# Without Cross Validations

## LR with past data - Iteration1 - Detrending

In [1]:
# import pandas as pd
# import numpy as np
# from pathlib import Path
# from sklearn.linear_model import Ridge, Lasso, ElasticNet
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import json
# import warnings
# warnings.filterwarnings('ignore')

# # Configuration
# processed_data_path = Path('../data/full-data/processed')
# output_path = Path('../data/full-data/final_temporal_models_regularization')
# output_path.mkdir(parents=True, exist_ok=True)

# mapping_df = pd.read_csv('../data/full-data/hotel_mapping.csv')
# hotel_list = mapping_df['masked_id'].tolist()

# # Temporal split configuration
# TRAIN_PCT = 0.60
# VAL_PCT = 0.20
# TEST_PCT = 0.20

# # Regularization parameters to test
# MODELS = {
#     'Ridge_1.0': Ridge(alpha=1.0, random_state=42),
#     'Ridge_10': Ridge(alpha=10.0, random_state=42),
#     'Ridge_100': Ridge(alpha=100.0, random_state=42),
#     'Lasso_0.1': Lasso(alpha=0.1, random_state=42, max_iter=10000),
#     'Lasso_1.0': Lasso(alpha=1.0, random_state=42, max_iter=10000),
#     'Lasso_10': Lasso(alpha=10.0, random_state=42, max_iter=10000),
#     'ElasticNet_0.1': ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42, max_iter=10000),
#     'ElasticNet_1.0': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=10000),
#     'ElasticNet_10': ElasticNet(alpha=10.0, l1_ratio=0.5, random_state=42, max_iter=10000)
# }

# print("="*100)
# print("LINEAR MODELS WITH MULTIPLE REGULARIZATION METHODS")
# print("="*100)
# print(f"Temporal Split: {TRAIN_PCT*100:.0f}% Train / {VAL_PCT*100:.0f}% Val / {TEST_PCT*100:.0f}% Test")
# print(f"Data Source: lagged_dataset.csv")
# print(f"Testing {len(MODELS)} regularization configurations:")
# for name in MODELS.keys():
#     print(f"  - {name}")
# print("-" * 100)

# def detrend_series(series, window=30):
#     """Detrend using rolling mean subtraction"""
#     if series.notna().sum() < window:
#         return series.fillna(0)
    
#     trend = series.rolling(window=window, min_periods=1, center=False).mean()
#     detrended = series - trend
#     return detrended.fillna(0)

# def prepare_features(df):
#     """Prepare all features with detrending"""
#     df_processed = df.copy()
    
#     # All competitor columns
#     all_comp_cols = [col for col in df.columns 
#                      if any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
#     comp_base_cols = [col for col in all_comp_cols if '_lag_' not in col]
#     comp_lag_cols = [col for col in all_comp_cols if '_lag_' in col]
    
#     # Detrend base competitor prices
#     for col in comp_base_cols:
#         if col in df_processed.columns:
#             df_processed[f'{col}_detrended'] = detrend_series(df_processed[col], window=30)
    
#     # Detrend competitor lags
#     for col in comp_lag_cols:
#         if col in df_processed.columns:
#             df_processed[f'{col}_detrended'] = detrend_series(df_processed[col], window=30)
    
#     # Detrend own price lags
#     for lag in [1, 2, 3, 4, 5]:
#         lag_col = f'base_rate_lag_{lag}'
#         if lag_col in df_processed.columns:
#             df_processed[f'{lag_col}_detrended'] = detrend_series(df_processed[lag_col], window=30)
    
#     # Feature columns
#     feature_cols = []
    
#     detrended_comp_base = [f'{col}_detrended' for col in comp_base_cols if f'{col}_detrended' in df_processed.columns]
#     feature_cols.extend(detrended_comp_base)
    
#     detrended_comp_lags = [f'{col}_detrended' for col in comp_lag_cols if f'{col}_detrended' in df_processed.columns]
#     feature_cols.extend(detrended_comp_lags)
    
#     detrended_own_lags = [f'base_rate_lag_{i}_detrended' for i in range(1, 6) 
#                          if f'base_rate_lag_{i}_detrended' in df_processed.columns]
#     feature_cols.extend(detrended_own_lags)
    
#     temporal_cols = ['day_of_week', 'month', 'is_weekend', 'is_peak_season', 
#                     'week_of_year', 'quarter', 'is_holiday_period', 'day_of_year',
#                     'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
#                     'sin_day_of_year', 'cos_day_of_year']
#     temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
#     feature_cols.extend(temporal_cols)
    
#     return df_processed, feature_cols

# def train_all_models_hotel(hotel_id):
#     """Train all regularization models for a single hotel"""
#     result = {
#         'hotel_id': hotel_id,
#         'status': 'training',
#         'models': {}
#     }
    
#     try:
#         data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        
#         if not data_file.exists():
#             result['status'] = 'missing_file'
#             return result
        
#         df = pd.read_csv(data_file)
#         df['date'] = pd.to_datetime(df['date'])
#         df = df.sort_values('date').reset_index(drop=True)
        
#         min_date = df['date'].min()
#         max_date = df['date'].max()
        
#         if len(df) < 150:
#             result['status'] = 'insufficient_data'
#             return result
        
#         # Prepare features
#         df_processed, feature_cols = prepare_features(df)
        
#         if len(feature_cols) == 0:
#             result['status'] = 'no_features'
#             return result
        
#         result['n_features'] = len(feature_cols)
        
#         # Split data
#         n_rows = len(df_processed)
#         train_end_idx = int(n_rows * TRAIN_PCT)
#         val_end_idx = int(n_rows * (TRAIN_PCT + VAL_PCT))
        
#         train_end_date = df_processed.iloc[train_end_idx - 1]['date']
#         val_end_date = df_processed.iloc[val_end_idx - 1]['date']
        
#         train_mask = df_processed['date'] <= train_end_date
#         val_mask = (df_processed['date'] > train_end_date) & (df_processed['date'] <= val_end_date)
#         test_mask = df_processed['date'] > val_end_date
        
#         df_train = df_processed[train_mask].copy()
#         df_val = df_processed[val_mask].copy()
#         df_test = df_processed[test_mask].copy()
        
#         if len(df_train) < 30 or len(df_val) < 10 or len(df_test) < 10:
#             result['status'] = 'insufficient_split_data'
#             return result
        
#         # Prepare data
#         X_train = df_train[feature_cols].fillna(0)
#         y_train = df_train['base_rate'].fillna(method='ffill').fillna(method='bfill')
        
#         X_val = df_val[feature_cols].fillna(0)
#         y_val = df_val['base_rate'].fillna(method='ffill').fillna(method='bfill')
        
#         X_test = df_test[feature_cols].fillna(0)
#         y_test = df_test['base_rate'].fillna(method='ffill').fillna(method='bfill')
        
#         # Standardize
#         scaler = StandardScaler()
#         X_train_scaled = scaler.fit_transform(X_train)
#         X_val_scaled = scaler.transform(X_val)
#         X_test_scaled = scaler.transform(X_test)
        
#         # Train all models
#         for model_name, model in MODELS.items():
#             try:
#                 model.fit(X_train_scaled, y_train)
                
#                 y_train_pred = model.predict(X_train_scaled)
#                 y_val_pred = model.predict(X_val_scaled)
#                 y_test_pred = model.predict(X_test_scaled)
                
#                 result['models'][model_name] = {
#                     'train_r2': float(r2_score(y_train, y_train_pred)),
#                     'train_rmse': float(np.sqrt(mean_squared_error(y_train, y_train_pred))),
#                     'val_r2': float(r2_score(y_val, y_val_pred)),
#                     'val_rmse': float(np.sqrt(mean_squared_error(y_val, y_val_pred))),
#                     'test_r2': float(r2_score(y_test, y_test_pred)),
#                     'test_rmse': float(np.sqrt(mean_squared_error(y_test, y_test_pred))),
#                     'test_mae': float(mean_absolute_error(y_test, y_test_pred))
#                 }
#             except Exception as e:
#                 result['models'][model_name] = {
#                     'error': str(e)
#                 }
        
#         result['status'] = 'success'
#         return result
        
#     except Exception as e:
#         result['status'] = 'error'
#         result['error'] = str(e)
#         return result

# # Train all hotels
# print("\nTraining all regularization models for all hotels...")
# print("-" * 100)

# all_results = {}
# for idx, hotel_id in enumerate(hotel_list, 1):
#     print(f"[{idx}/{len(hotel_list)}] {hotel_id}...", end='\r')
#     result = train_all_models_hotel(hotel_id)
#     all_results[hotel_id] = result

# successful = [h for h, r in all_results.items() if r['status'] == 'success']
# print(f"\n\nSuccessfully trained: {len(successful)}/{len(hotel_list)} hotels")

# # Save results
# with open(output_path / 'regularization_comparison_results.json', 'w') as f:
#     json.dump(all_results, f, indent=2)

# # Generate comparison report
# print("\n" + "="*100)
# print("REGULARIZATION COMPARISON REPORT")
# print("="*100)

# if len(successful) > 0:
#     # Compare models
#     model_comparison = {}
    
#     for model_name in MODELS.keys():
#         test_r2s = []
#         val_r2s = []
#         test_rmses = []
        
#         for hotel_id in successful:
#             if model_name in all_results[hotel_id]['models']:
#                 model_result = all_results[hotel_id]['models'][model_name]
#                 if 'test_r2' in model_result:
#                     test_r2s.append(model_result['test_r2'])
#                     val_r2s.append(model_result['val_r2'])
#                     test_rmses.append(model_result['test_rmse'])
        
#         if len(test_r2s) > 0:
#             model_comparison[model_name] = {
#                 'median_test_r2': np.median(test_r2s),
#                 'mean_test_r2': np.mean(test_r2s),
#                 'median_val_r2': np.median(val_r2s),
#                 'median_test_rmse': np.median(test_rmses),
#                 'n_excellent': sum(1 for r in test_r2s if r > 0.70),
#                 'n_good': sum(1 for r in test_r2s if 0.40 < r <= 0.70),
#                 'n_poor': sum(1 for r in test_r2s if r <= 0.20)
#             }
    
#     # Sort by median test R²
#     sorted_models = sorted(model_comparison.items(), key=lambda x: x[1]['median_test_r2'], reverse=True)
    
#     print("\nModel Comparison (sorted by Median Test R²):")
#     print("-" * 100)
#     print(f"{'Model':<20} {'Med Test R²':<15} {'Med Val R²':<15} {'Med RMSE':<12} {'Excellent':<12} {'Good':<8} {'Poor':<8}")
#     print("-" * 100)
    
#     for model_name, metrics in sorted_models:
#         print(f"{model_name:<20} {metrics['median_test_r2']:>14.4f} {metrics['median_val_r2']:>14.4f} "
#               f"{metrics['median_test_rmse']:>11.2f} {metrics['n_excellent']:>11} "
#               f"{metrics['n_good']:>7} {metrics['n_poor']:>7}")
    
#     # Best model
#     best_model = sorted_models[0][0]
#     print(f"\nBest Model: {best_model}")
#     print(f"Median Test R²: {sorted_models[0][1]['median_test_r2']:.4f}")
    
#     # Create summary CSV
#     summary_data = []
#     for hotel_id in successful:
#         hotel_row = {'hotel_id': hotel_id}
#         for model_name in MODELS.keys():
#             if model_name in all_results[hotel_id]['models']:
#                 model_result = all_results[hotel_id]['models'][model_name]
#                 if 'test_r2' in model_result:
#                     hotel_row[f'{model_name}_test_r2'] = model_result['test_r2']
#                     hotel_row[f'{model_name}_val_r2'] = model_result['val_r2']
#         summary_data.append(hotel_row)
    
#     summary_df = pd.DataFrame(summary_data)
#     summary_df.to_csv(output_path / 'regularization_comparison_summary.csv', index=False)
    
#     print(f"\nFiles saved to: {output_path}/")

# print("\n" + "="*100)
# print("SUMMARY")
# print("="*100)
# print(f"Tested {len(MODELS)} regularization configurations")
# print(f"Successfully trained: {len(successful)}/{len(hotel_list)} hotels")
# print("="*100)

## LR with past data - Iteration2 - Log transformation 

In [2]:
# import pandas as pd
# import numpy as np
# from pathlib import Path
# from sklearn.linear_model import Ridge, Lasso, ElasticNet
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import json
# import warnings
# warnings.filterwarnings('ignore')

# # Configuration
# processed_data_path = Path('../data/full-data/processed')
# output_path = Path('../data/full-data/final_log_strong_regularization')
# output_path.mkdir(parents=True, exist_ok=True)

# mapping_df = pd.read_csv('../data/full-data/hotel_mapping.csv')
# hotel_list = mapping_df['masked_id'].tolist()

# # Temporal split configuration
# TRAIN_PCT = 0.60
# VAL_PCT = 0.20
# TEST_PCT = 0.20

# # Best regularization models from previous analysis
# MODELS = {
#     'Ridge_100': Ridge(alpha=100.0, random_state=42),
#     'ElasticNet_1.0': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=10000),
#     'Lasso_1.0': Lasso(alpha=1.0, random_state=42, max_iter=10000)
# }

# print("="*100)
# print("LINEAR MODELS WITH LOG TRANSFORMATION + STRONG REGULARIZATION")
# print("="*100)
# print(f"Temporal Split: {TRAIN_PCT*100:.0f}% Train / {VAL_PCT*100:.0f}% Val / {TEST_PCT*100:.0f}% Test")
# print(f"Data Source: lagged_dataset.csv (full feature set)")
# print(f"Transformation: log1p (log(1 + x))")
# print(f"Testing 3 best regularization methods:")
# for name in MODELS.keys():
#     print(f"  - {name}")
# print("-" * 100)

# def detrend_series_log(series, window=30):
#     """Detrend in log space using rolling mean subtraction"""
#     if series.notna().sum() < window:
#         return series.fillna(0)
    
#     # Apply log1p first
#     log_series = np.log1p(series.clip(lower=0))
    
#     # Detrend in log space
#     trend = log_series.rolling(window=window, min_periods=1, center=False).mean()
#     detrended = log_series - trend
    
#     return detrended.fillna(0)

# def prepare_features_log(df):
#     """Prepare all features with log transformation and detrending"""
#     df_processed = df.copy()
    
#     # All competitor columns
#     all_comp_cols = [col for col in df.columns 
#                      if any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
#     comp_base_cols = [col for col in all_comp_cols if '_lag_' not in col]
#     comp_lag_cols = [col for col in all_comp_cols if '_lag_' in col]
    
#     # Log transform + detrend base competitor prices
#     for col in comp_base_cols:
#         if col in df_processed.columns:
#             df_processed[f'{col}_log_detrended'] = detrend_series_log(df_processed[col], window=30)
    
#     # Log transform + detrend competitor lags
#     for col in comp_lag_cols:
#         if col in df_processed.columns:
#             df_processed[f'{col}_log_detrended'] = detrend_series_log(df_processed[col], window=30)
    
#     # Log transform + detrend own price lags
#     for lag in [1, 2, 3, 4, 5]:
#         lag_col = f'base_rate_lag_{lag}'
#         if lag_col in df_processed.columns:
#             df_processed[f'{lag_col}_log_detrended'] = detrend_series_log(df_processed[lag_col], window=30)
    
#     # Feature columns
#     feature_cols = []
    
#     log_detrended_comp_base = [f'{col}_log_detrended' for col in comp_base_cols 
#                                 if f'{col}_log_detrended' in df_processed.columns]
#     feature_cols.extend(log_detrended_comp_base)
    
#     log_detrended_comp_lags = [f'{col}_log_detrended' for col in comp_lag_cols 
#                                 if f'{col}_log_detrended' in df_processed.columns]
#     feature_cols.extend(log_detrended_comp_lags)
    
#     log_detrended_own_lags = [f'base_rate_lag_{i}_log_detrended' for i in range(1, 6) 
#                               if f'base_rate_lag_{i}_log_detrended' in df_processed.columns]
#     feature_cols.extend(log_detrended_own_lags)
    
#     temporal_cols = ['day_of_week', 'month', 'is_weekend', 'is_peak_season', 
#                     'week_of_year', 'quarter', 'is_holiday_period', 'day_of_year',
#                     'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
#                     'sin_day_of_year', 'cos_day_of_year']
#     temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
#     feature_cols.extend(temporal_cols)
    
#     return df_processed, feature_cols

# def train_all_models_hotel(hotel_id):
#     """Train all log-transformed models for a single hotel"""
#     result = {
#         'hotel_id': hotel_id,
#         'status': 'training',
#         'models': {}
#     }
    
#     try:
#         data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        
#         if not data_file.exists():
#             result['status'] = 'missing_file'
#             return result
        
#         df = pd.read_csv(data_file)
#         df['date'] = pd.to_datetime(df['date'])
#         df = df.sort_values('date').reset_index(drop=True)
        
#         if len(df) < 150:
#             result['status'] = 'insufficient_data'
#             return result
        
#         # Prepare features with log transformation
#         df_processed, feature_cols = prepare_features_log(df)
        
#         if len(feature_cols) == 0:
#             result['status'] = 'no_features'
#             return result
        
#         result['n_features'] = len(feature_cols)
        
#         # Split data (percentage-based)
#         n_rows = len(df_processed)
#         train_end_idx = int(n_rows * TRAIN_PCT)
#         val_end_idx = int(n_rows * (TRAIN_PCT + VAL_PCT))
        
#         train_end_date = df_processed.iloc[train_end_idx - 1]['date']
#         val_end_date = df_processed.iloc[val_end_idx - 1]['date']
        
#         train_mask = df_processed['date'] <= train_end_date
#         val_mask = (df_processed['date'] > train_end_date) & (df_processed['date'] <= val_end_date)
#         test_mask = df_processed['date'] > val_end_date
        
#         df_train = df_processed[train_mask].copy()
#         df_val = df_processed[val_mask].copy()
#         df_test = df_processed[test_mask].copy()
        
#         if len(df_train) < 30 or len(df_val) < 10 or len(df_test) < 10:
#             result['status'] = 'insufficient_split_data'
#             return result
        
#         # Store split info
#         result['split_info'] = {
#             'train_start': str(df_train['date'].min()),
#             'train_end': str(df_train['date'].max()),
#             'train_rows': len(df_train),
#             'val_start': str(df_val['date'].min()),
#             'val_end': str(df_val['date'].max()),
#             'val_rows': len(df_val),
#             'test_start': str(df_test['date'].min()),
#             'test_end': str(df_test['date'].max()),
#             'test_rows': len(df_test)
#         }
        
#         # Prepare data
#         X_train = df_train[feature_cols].fillna(0)
#         # Target in log space
#         y_train_original = df_train['base_rate'].fillna(method='ffill').fillna(method='bfill')
#         y_train = np.log1p(y_train_original.clip(lower=0))
        
#         X_val = df_val[feature_cols].fillna(0)
#         y_val_original = df_val['base_rate'].fillna(method='ffill').fillna(method='bfill')
#         y_val = np.log1p(y_val_original.clip(lower=0))
        
#         X_test = df_test[feature_cols].fillna(0)
#         y_test_original = df_test['base_rate'].fillna(method='ffill').fillna(method='bfill')
#         y_test = np.log1p(y_test_original.clip(lower=0))
        
#         # Standardize features
#         scaler = StandardScaler()
#         X_train_scaled = scaler.fit_transform(X_train)
#         X_val_scaled = scaler.transform(X_val)
#         X_test_scaled = scaler.transform(X_test)
        
#         # Train all models
#         for model_name, model in MODELS.items():
#             try:
#                 model.fit(X_train_scaled, y_train)
                
#                 # Predictions in log space
#                 y_train_pred_log = model.predict(X_train_scaled)
#                 y_val_pred_log = model.predict(X_val_scaled)
#                 y_test_pred_log = model.predict(X_test_scaled)
                
#                 # Transform back to original space
#                 y_train_pred = np.expm1(y_train_pred_log)
#                 y_val_pred = np.expm1(y_val_pred_log)
#                 y_test_pred = np.expm1(y_test_pred_log)
                
#                 # Calculate metrics in original space
#                 result['models'][model_name] = {
#                     'train_r2': float(r2_score(y_train_original, y_train_pred)),
#                     'train_rmse': float(np.sqrt(mean_squared_error(y_train_original, y_train_pred))),
#                     'val_r2': float(r2_score(y_val_original, y_val_pred)),
#                     'val_rmse': float(np.sqrt(mean_squared_error(y_val_original, y_val_pred))),
#                     'test_r2': float(r2_score(y_test_original, y_test_pred)),
#                     'test_rmse': float(np.sqrt(mean_squared_error(y_test_original, y_test_pred))),
#                     'test_mae': float(mean_absolute_error(y_test_original, y_test_pred)),
#                     'train_test_gap': float(r2_score(y_train_original, y_train_pred) - r2_score(y_test_original, y_test_pred))
#                 }
                
#                 # Save predictions for best model (Ridge_100)
#                 if model_name == 'Ridge_100':
#                     predictions_df = pd.DataFrame({
#                         'date': df_test['date'],
#                         'actual_price': y_test_original,
#                         'predicted_price': y_test_pred,
#                         'error': y_test_original - y_test_pred,
#                         'abs_error': np.abs(y_test_original - y_test_pred),
#                         'pct_error': 100 * (y_test_original - y_test_pred) / y_test_original
#                     })
#                     predictions_df.to_csv(output_path / f'{hotel_id}_log_ridge100_predictions.csv', index=False)
                
#             except Exception as e:
#                 result['models'][model_name] = {
#                     'error': str(e)
#                 }
        
#         result['status'] = 'success'
#         return result
        
#     except Exception as e:
#         result['status'] = 'error'
#         result['error'] = str(e)
#         return result

# # Train all hotels
# print("\nTraining log-transformed models with strong regularization...")
# print("-" * 100)

# all_results = {}
# for idx, hotel_id in enumerate(hotel_list, 1):
#     print(f"[{idx}/{len(hotel_list)}] {hotel_id}...", end='\r')
#     result = train_all_models_hotel(hotel_id)
#     all_results[hotel_id] = result

# successful = [h for h, r in all_results.items() if r['status'] == 'success']
# print(f"\n\nSuccessfully trained: {len(successful)}/{len(hotel_list)} hotels")

# # Save results
# with open(output_path / 'log_strong_regularization_results.json', 'w') as f:
#     json.dump(all_results, f, indent=2)

# # Generate comparison report
# print("\n" + "="*100)
# print("LOG TRANSFORMATION + STRONG REGULARIZATION REPORT")
# print("="*100)

# if len(successful) > 0:
#     # Compare models
#     model_comparison = {}
    
#     for model_name in MODELS.keys():
#         test_r2s = []
#         val_r2s = []
#         train_r2s = []
#         gaps = []
        
#         for hotel_id in successful:
#             if model_name in all_results[hotel_id]['models']:
#                 model_result = all_results[hotel_id]['models'][model_name]
#                 if 'test_r2' in model_result:
#                     test_r2s.append(model_result['test_r2'])
#                     val_r2s.append(model_result['val_r2'])
#                     train_r2s.append(model_result['train_r2'])
#                     gaps.append(model_result['train_test_gap'])
        
#         if len(test_r2s) > 0:
#             test_r2s_sorted = sorted(test_r2s)
#             model_comparison[model_name] = {
#                 'median_test_r2': np.median(test_r2s),
#                 'mean_test_r2': np.mean(test_r2s),
#                 'median_val_r2': np.median(val_r2s),
#                 'median_train_r2': np.median(train_r2s),
#                 'median_gap': np.median(gaps),
#                 'n_excellent': sum(1 for r in test_r2s if r > 0.70),
#                 'n_good': sum(1 for r in test_r2s if 0.40 < r <= 0.70),
#                 'n_moderate': sum(1 for r in test_r2s if 0.20 < r <= 0.40),
#                 'n_poor': sum(1 for r in test_r2s if r <= 0.20)
#             }
    
#     # Sort by median test R²
#     sorted_models = sorted(model_comparison.items(), key=lambda x: x[1]['median_test_r2'], reverse=True)
    
#     print("\nModel Comparison (sorted by Median Test R²):")
#     print("-" * 100)
#     print(f"{'Model':<20} {'Med Test R²':<15} {'Med Val R²':<15} {'Med Gap':<12} {'Excellent':<12} {'Good':<8} {'Poor':<8}")
#     print("-" * 100)
    
#     for model_name, metrics in sorted_models:
#         print(f"{model_name:<20} {metrics['median_test_r2']:>14.4f} {metrics['median_val_r2']:>14.4f} "
#               f"{metrics['median_gap']:>11.4f} {metrics['n_excellent']:>11} "
#               f"{metrics['n_good']:>7} {metrics['n_poor']:>7}")
    
#     # Best model
#     best_model = sorted_models[0][0]
#     print(f"\nBest Model: {best_model}")
#     print(f"Median Test R²: {sorted_models[0][1]['median_test_r2']:.4f}")
#     print(f"Overfitting Gap: {sorted_models[0][1]['median_gap']:.4f}")
    
#     # Top performers
#     print("\n" + "="*100)
#     print("TOP 10 HOTELS (by Ridge_100 test R²)")
#     print("="*100)
    
#     top_hotels = []
#     for hotel_id in successful:
#         if 'Ridge_100' in all_results[hotel_id]['models']:
#             model_result = all_results[hotel_id]['models']['Ridge_100']
#             if 'test_r2' in model_result:
#                 top_hotels.append({
#                     'hotel_id': hotel_id,
#                     'test_r2': model_result['test_r2'],
#                     'val_r2': model_result['val_r2'],
#                     'train_r2': model_result['train_r2'],
#                     'gap': model_result['train_test_gap'],
#                     'test_rmse': model_result['test_rmse']
#                 })
    
#     top_hotels.sort(key=lambda x: x['test_r2'], reverse=True)
    
#     print(f"{'Hotel':<12} {'Test R²':<12} {'Val R²':<12} {'Train R²':<12} {'Gap':<12} {'RMSE':<12}")
#     print("-" * 100)
#     for h in top_hotels[:10]:
#         print(f"{h['hotel_id']:<12} {h['test_r2']:>11.4f} {h['val_r2']:>11.4f} "
#               f"{h['train_r2']:>11.4f} {h['gap']:>11.4f} {h['test_rmse']:>11.2f}")
    
#     # Create summary CSV
#     summary_data = []
#     for hotel_id in successful:
#         hotel_row = {'hotel_id': hotel_id}
#         for model_name in MODELS.keys():
#             if model_name in all_results[hotel_id]['models']:
#                 model_result = all_results[hotel_id]['models'][model_name]
#                 if 'test_r2' in model_result:
#                     hotel_row[f'{model_name}_test_r2'] = model_result['test_r2']
#                     hotel_row[f'{model_name}_val_r2'] = model_result['val_r2']
#                     hotel_row[f'{model_name}_gap'] = model_result['train_test_gap']
#         summary_data.append(hotel_row)
    
#     summary_df = pd.DataFrame(summary_data)
#     summary_df.to_csv(output_path / 'log_strong_regularization_summary.csv', index=False)
    
#     print(f"\nFiles saved to: {output_path}/")

# print("\n" + "="*100)
# print("SUMMARY")
# print("="*100)
# print(f"Successfully trained: {len(successful)}/{len(hotel_list)} hotels")
# print(f"Using log1p transformation + 30-day detrending in log space")
# print(f"Testing 3 best regularization methods from previous analysis")
# print("="*100)

## LR with past data - Iteration3 - Quantile Normalization

In [3]:
# import pandas as pd
# import numpy as np
# from pathlib import Path
# from sklearn.linear_model import Ridge, Lasso, ElasticNet
# from sklearn.preprocessing import QuantileTransformer, StandardScaler
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import json
# import warnings
# warnings.filterwarnings('ignore')

# # Configuration
# processed_data_path = Path('../data/full-data/processed')
# output_path = Path('../data/full-data/final_quantile_normalization')
# output_path.mkdir(parents=True, exist_ok=True)

# mapping_df = pd.read_csv('../data/full-data/hotel_mapping.csv')
# hotel_list = mapping_df['masked_id'].tolist()

# # Temporal split configuration
# TRAIN_PCT = 0.60
# VAL_PCT = 0.20
# TEST_PCT = 0.20

# # Best regularization models
# MODELS = {
#     'Ridge_100': Ridge(alpha=100.0, random_state=42),
#     'Ridge_10': Ridge(alpha=10.0, random_state=42),
#     'ElasticNet_1.0': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=10000),
#     'Lasso_1.0': Lasso(alpha=1.0, random_state=42, max_iter=10000)
# }

# print("="*100)
# print("LINEAR MODELS WITH QUANTILE NORMALIZATION + STRONG REGULARIZATION")
# print("="*100)
# print(f"Temporal Split: {TRAIN_PCT*100:.0f}% Train / {VAL_PCT*100:.0f}% Val / {TEST_PCT*100:.0f}% Test")
# print(f"Data Source: lagged_dataset.csv (full feature set)")
# print(f"Transformation: QuantileTransformer (output='normal', robust to outliers)")
# print(f"Testing 4 regularization methods:")
# for name in MODELS.keys():
#     print(f"  - {name}")
# print("\nQuantile Normalization Benefits:")
# print("  - Maps features to normal distribution (Gaussian)")
# print("  - Handles outliers robustly")
# print("  - Makes features comparable across different scales")
# print("  - Non-linear transformation preserving rank order")
# print("-" * 100)

# def detrend_series(series, window=30):
#     """Detrend using rolling mean subtraction (in original space)"""
#     if series.notna().sum() < window:
#         return series.fillna(0)
    
#     trend = series.rolling(window=window, min_periods=1, center=False).mean()
#     detrended = series - trend
#     return detrended.fillna(0)

# def prepare_features_quantile(df):
#     """Prepare all features with detrending (quantile transform applied later)"""
#     df_processed = df.copy()
    
#     # All competitor columns
#     all_comp_cols = [col for col in df.columns 
#                      if any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
#     comp_base_cols = [col for col in all_comp_cols if '_lag_' not in col]
#     comp_lag_cols = [col for col in all_comp_cols if '_lag_' in col]
    
#     # Detrend base competitor prices
#     for col in comp_base_cols:
#         if col in df_processed.columns:
#             df_processed[f'{col}_detrended'] = detrend_series(df_processed[col], window=30)
    
#     # Detrend competitor lags
#     for col in comp_lag_cols:
#         if col in df_processed.columns:
#             df_processed[f'{col}_detrended'] = detrend_series(df_processed[col], window=30)
    
#     # Detrend own price lags
#     for lag in [1, 2, 3, 4, 5]:
#         lag_col = f'base_rate_lag_{lag}'
#         if lag_col in df_processed.columns:
#             df_processed[f'{lag_col}_detrended'] = detrend_series(df_processed[lag_col], window=30)
    
#     # Feature columns
#     feature_cols = []
    
#     detrended_comp_base = [f'{col}_detrended' for col in comp_base_cols 
#                           if f'{col}_detrended' in df_processed.columns]
#     feature_cols.extend(detrended_comp_base)
    
#     detrended_comp_lags = [f'{col}_detrended' for col in comp_lag_cols 
#                           if f'{col}_detrended' in df_processed.columns]
#     feature_cols.extend(detrended_comp_lags)
    
#     detrended_own_lags = [f'base_rate_lag_{i}_detrended' for i in range(1, 6) 
#                          if f'base_rate_lag_{i}_detrended' in df_processed.columns]
#     feature_cols.extend(detrended_own_lags)
    
#     temporal_cols = ['day_of_week', 'month', 'is_weekend', 'is_peak_season', 
#                     'week_of_year', 'quarter', 'is_holiday_period', 'day_of_year',
#                     'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
#                     'sin_day_of_year', 'cos_day_of_year']
#     temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
#     feature_cols.extend(temporal_cols)
    
#     return df_processed, feature_cols

# def train_all_models_hotel(hotel_id):
#     """Train all quantile-normalized models for a single hotel"""
#     result = {
#         'hotel_id': hotel_id,
#         'status': 'training',
#         'models': {}
#     }
    
#     try:
#         data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        
#         if not data_file.exists():
#             result['status'] = 'missing_file'
#             return result
        
#         df = pd.read_csv(data_file)
#         df['date'] = pd.to_datetime(df['date'])
#         df = df.sort_values('date').reset_index(drop=True)
        
#         if len(df) < 150:
#             result['status'] = 'insufficient_data'
#             return result
        
#         # Prepare features with detrending
#         df_processed, feature_cols = prepare_features_quantile(df)
        
#         if len(feature_cols) == 0:
#             result['status'] = 'no_features'
#             return result
        
#         result['n_features'] = len(feature_cols)
        
#         # Split data (percentage-based)
#         n_rows = len(df_processed)
#         train_end_idx = int(n_rows * TRAIN_PCT)
#         val_end_idx = int(n_rows * (TRAIN_PCT + VAL_PCT))
        
#         train_end_date = df_processed.iloc[train_end_idx - 1]['date']
#         val_end_date = df_processed.iloc[val_end_idx - 1]['date']
        
#         train_mask = df_processed['date'] <= train_end_date
#         val_mask = (df_processed['date'] > train_end_date) & (df_processed['date'] <= val_end_date)
#         test_mask = df_processed['date'] > val_end_date
        
#         df_train = df_processed[train_mask].copy()
#         df_val = df_processed[val_mask].copy()
#         df_test = df_processed[test_mask].copy()
        
#         if len(df_train) < 30 or len(df_val) < 10 or len(df_test) < 10:
#             result['status'] = 'insufficient_split_data'
#             return result
        
#         # Store split info
#         result['split_info'] = {
#             'train_start': str(df_train['date'].min()),
#             'train_end': str(df_train['date'].max()),
#             'train_rows': len(df_train),
#             'val_start': str(df_val['date'].min()),
#             'val_end': str(df_val['date'].max()),
#             'val_rows': len(df_val),
#             'test_start': str(df_test['date'].min()),
#             'test_end': str(df_test['date'].max()),
#             'test_rows': len(df_test)
#         }
        
#         # Prepare data
#         X_train = df_train[feature_cols].fillna(0)
#         y_train = df_train['base_rate'].fillna(method='ffill').fillna(method='bfill')
        
#         X_val = df_val[feature_cols].fillna(0)
#         y_val = df_val['base_rate'].fillna(method='ffill').fillna(method='bfill')
        
#         X_test = df_test[feature_cols].fillna(0)
#         y_test = df_test['base_rate'].fillna(method='ffill').fillna(method='bfill')
        
#         # Apply Quantile Transformation
#         # output='normal' maps to Gaussian distribution
#         # n_quantiles should be <= n_samples, use min of 1000 or train size
#         n_quantiles = min(1000, len(X_train))
        
#         quantile_transformer = QuantileTransformer(
#             n_quantiles=n_quantiles,
#             output_distribution='normal',
#             random_state=42
#         )
        
#         # Fit on training data only
#         X_train_quantile = quantile_transformer.fit_transform(X_train)
#         X_val_quantile = quantile_transformer.transform(X_val)
#         X_test_quantile = quantile_transformer.transform(X_test)
        
#         # Train all models
#         for model_name, model in MODELS.items():
#             try:
#                 model.fit(X_train_quantile, y_train)
                
#                 # Predictions
#                 y_train_pred = model.predict(X_train_quantile)
#                 y_val_pred = model.predict(X_val_quantile)
#                 y_test_pred = model.predict(X_test_quantile)
                
#                 # Calculate metrics
#                 result['models'][model_name] = {
#                     'train_r2': float(r2_score(y_train, y_train_pred)),
#                     'train_rmse': float(np.sqrt(mean_squared_error(y_train, y_train_pred))),
#                     'val_r2': float(r2_score(y_val, y_val_pred)),
#                     'val_rmse': float(np.sqrt(mean_squared_error(y_val, y_val_pred))),
#                     'test_r2': float(r2_score(y_test, y_test_pred)),
#                     'test_rmse': float(np.sqrt(mean_squared_error(y_test, y_test_pred))),
#                     'test_mae': float(mean_absolute_error(y_test, y_test_pred)),
#                     'train_test_gap': float(r2_score(y_train, y_train_pred) - r2_score(y_test, y_test_pred))
#                 }
                
#                 # Save predictions for best model (Ridge_100)
#                 if model_name == 'Ridge_100':
#                     predictions_df = pd.DataFrame({
#                         'date': df_test['date'],
#                         'actual_price': y_test,
#                         'predicted_price': y_test_pred,
#                         'error': y_test - y_test_pred,
#                         'abs_error': np.abs(y_test - y_test_pred),
#                         'pct_error': 100 * (y_test - y_test_pred) / y_test
#                     })
#                     predictions_df.to_csv(output_path / f'{hotel_id}_quantile_ridge100_predictions.csv', index=False)
                
#             except Exception as e:
#                 result['models'][model_name] = {
#                     'error': str(e)
#                 }
        
#         result['status'] = 'success'
#         return result
        
#     except Exception as e:
#         result['status'] = 'error'
#         result['error'] = str(e)
#         return result

# # Train all hotels
# print("\nTraining quantile-normalized models with strong regularization...")
# print("-" * 100)

# all_results = {}
# for idx, hotel_id in enumerate(hotel_list, 1):
#     print(f"[{idx}/{len(hotel_list)}] {hotel_id}...", end='\r')
#     result = train_all_models_hotel(hotel_id)
#     all_results[hotel_id] = result

# successful = [h for h, r in all_results.items() if r['status'] == 'success']
# print(f"\n\nSuccessfully trained: {len(successful)}/{len(hotel_list)} hotels")

# # Save results
# with open(output_path / 'quantile_normalization_results.json', 'w') as f:
#     json.dump(all_results, f, indent=2)

# # Generate comparison report
# print("\n" + "="*100)
# print("QUANTILE NORMALIZATION + STRONG REGULARIZATION REPORT")
# print("="*100)

# if len(successful) > 0:
#     # Compare models
#     model_comparison = {}
    
#     for model_name in MODELS.keys():
#         test_r2s = []
#         val_r2s = []
#         train_r2s = []
#         gaps = []
        
#         for hotel_id in successful:
#             if model_name in all_results[hotel_id]['models']:
#                 model_result = all_results[hotel_id]['models'][model_name]
#                 if 'test_r2' in model_result:
#                     test_r2s.append(model_result['test_r2'])
#                     val_r2s.append(model_result['val_r2'])
#                     train_r2s.append(model_result['train_r2'])
#                     gaps.append(model_result['train_test_gap'])
        
#         if len(test_r2s) > 0:
#             test_r2s_sorted = sorted(test_r2s)
#             model_comparison[model_name] = {
#                 'median_test_r2': np.median(test_r2s),
#                 'mean_test_r2': np.mean(test_r2s),
#                 'median_val_r2': np.median(val_r2s),
#                 'median_train_r2': np.median(train_r2s),
#                 'median_gap': np.median(gaps),
#                 'n_excellent': sum(1 for r in test_r2s if r > 0.70),
#                 'n_good': sum(1 for r in test_r2s if 0.40 < r <= 0.70),
#                 'n_moderate': sum(1 for r in test_r2s if 0.20 < r <= 0.40),
#                 'n_poor': sum(1 for r in test_r2s if r <= 0.20)
#             }
    
#     # Sort by median test R²
#     sorted_models = sorted(model_comparison.items(), key=lambda x: x[1]['median_test_r2'], reverse=True)
    
#     print("\nModel Comparison (sorted by Median Test R²):")
#     print("-" * 100)
#     print(f"{'Model':<20} {'Med Test R²':<15} {'Med Val R²':<15} {'Med Gap':<12} {'Excellent':<12} {'Good':<8} {'Poor':<8}")
#     print("-" * 100)
    
#     for model_name, metrics in sorted_models:
#         print(f"{model_name:<20} {metrics['median_test_r2']:>14.4f} {metrics['median_val_r2']:>14.4f} "
#               f"{metrics['median_gap']:>11.4f} {metrics['n_excellent']:>11} "
#               f"{metrics['n_good']:>7} {metrics['n_poor']:>7}")
    
#     # Best model
#     best_model = sorted_models[0][0]
#     print(f"\nBest Model: {best_model}")
#     print(f"Median Test R²: {sorted_models[0][1]['median_test_r2']:.4f}")
#     print(f"Overfitting Gap: {sorted_models[0][1]['median_gap']:.4f}")
    
#     # Top performers
#     print("\n" + "="*100)
#     print("TOP 10 HOTELS (by Ridge_100 test R²)")
#     print("="*100)
    
#     top_hotels = []
#     for hotel_id in successful:
#         if 'Ridge_100' in all_results[hotel_id]['models']:
#             model_result = all_results[hotel_id]['models']['Ridge_100']
#             if 'test_r2' in model_result:
#                 top_hotels.append({
#                     'hotel_id': hotel_id,
#                     'test_r2': model_result['test_r2'],
#                     'val_r2': model_result['val_r2'],
#                     'train_r2': model_result['train_r2'],
#                     'gap': model_result['train_test_gap'],
#                     'test_rmse': model_result['test_rmse']
#                 })
    
#     top_hotels.sort(key=lambda x: x['test_r2'], reverse=True)
    
#     print(f"{'Hotel':<12} {'Test R²':<12} {'Val R²':<12} {'Train R²':<12} {'Gap':<12} {'RMSE':<12}")
#     print("-" * 100)
#     for h in top_hotels[:10]:
#         print(f"{h['hotel_id']:<12} {h['test_r2']:>11.4f} {h['val_r2']:>11.4f} "
#               f"{h['train_r2']:>11.4f} {h['gap']:>11.4f} {h['test_rmse']:>11.2f}")
    
#     # Create summary CSV
#     summary_data = []
#     for hotel_id in successful:
#         hotel_row = {'hotel_id': hotel_id}
#         for model_name in MODELS.keys():
#             if model_name in all_results[hotel_id]['models']:
#                 model_result = all_results[hotel_id]['models'][model_name]
#                 if 'test_r2' in model_result:
#                     hotel_row[f'{model_name}_test_r2'] = model_result['test_r2']
#                     hotel_row[f'{model_name}_val_r2'] = model_result['val_r2']
#                     hotel_row[f'{model_name}_gap'] = model_result['train_test_gap']
#         summary_data.append(hotel_row)
    
#     summary_df = pd.DataFrame(summary_data)
#     summary_df.to_csv(output_path / 'quantile_normalization_summary.csv', index=False)
    
#     print(f"\nFiles saved to: {output_path}/")

# print("\n" + "="*100)
# print("SUMMARY")
# print("="*100)
# print(f"Successfully trained: {len(successful)}/{len(hotel_list)} hotels")
# print(f"Using QuantileTransformer (output='normal') + 30-day detrending")
# print(f"Testing 4 regularization methods")
# print("\nQuantile Normalization transforms features to Gaussian distribution")
# print("This handles outliers and makes features comparable across scales")
# print("="*100)

# With Cross Validation

## LR with past data - Iteration1 - Detrending

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import warnings
warnings.filterwarnings('ignore')

"""
LINEAR + DETRENDING - OUT-OF-SAMPLE CROSS-VALIDATION
=====================================================
ONLY competitor prices + temporal features (NO own price lags)
Expanding window strategy
"""

# Configuration
processed_data_path = Path('../data/full-data/processed')
output_path = Path('../data/full-data/linear_detrending_oos_cv_clean')
output_path.mkdir(parents=True, exist_ok=True)

mapping_df = pd.read_csv('../data/full-data/hotel_mapping.csv')
hotel_list = mapping_df['masked_id'].tolist()

# CV Configuration
MIN_TRAIN_DAYS = 300
TEST_WINDOW = 50
TREND_WINDOW = 30
ALPHA = 100.0

print("="*140)
print("LINEAR REGRESSION + DETRENDING - OUT-OF-SAMPLE CROSS-VALIDATION")
print("="*140)
print(f"Model: Ridge Regression (α={ALPHA})")
print(f"Preprocessing: 30-day detrending + StandardScaler")
print(f"Features: Competitor prices + temporal features ONLY (NO own price lags)")
print(f"CV Strategy: Expanding window (min {MIN_TRAIN_DAYS} days, test window {TEST_WINDOW} days)")
print(f"Testing {len(hotel_list)} hotels")
print("-" * 140)

def detrend_series(series, window=30):
    """Detrend using rolling mean subtraction"""
    trend = series.rolling(window=window, min_periods=1, center=False).mean()
    detrended = series - trend
    return detrended, trend

def prepare_features(df):
    """Prepare features - ONLY competitor prices + temporal, NO own lags"""
    df_processed = df.copy()
    
    # Get all competitor columns (base prices and lags)
    all_comp_cols = [col for col in df.columns 
                     if any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
    comp_base_cols = [col for col in all_comp_cols if '_lag_' not in col]
    comp_lag_cols = [col for col in all_comp_cols if '_lag_' in col]
    
    # Detrend base competitor prices
    for col in comp_base_cols:
        if col in df_processed.columns:
            detrended, _ = detrend_series(df_processed[col], window=TREND_WINDOW)
            df_processed[f'{col}_detrended'] = detrended
    
    # Detrend competitor lags
    for col in comp_lag_cols:
        if col in df_processed.columns:
            detrended, _ = detrend_series(df_processed[col], window=TREND_WINDOW)
            df_processed[f'{col}_detrended'] = detrended
    
    # Build feature list - ONLY COMPETITORS + TEMPORAL
    feature_cols = []
    
    # Detrended competitor lags
    detrended_comp_lags = [f'{col}_detrended' for col in comp_lag_cols 
                          if f'{col}_detrended' in df_processed.columns]
    feature_cols.extend(detrended_comp_lags)
    
    # Temporal features only
    temporal_cols = ['day_of_week', 'month', 'is_weekend', 'is_peak_season', 
                    'week_of_year', 'quarter', 'is_holiday_period', 'day_of_year',
                    'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
                    'sin_day_of_year', 'cos_day_of_year']
    temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
    feature_cols.extend(temporal_cols)
    
    # Target - detrend base_rate
    y_original = df_processed['base_rate']
    y_detrended, y_trend = detrend_series(y_original, window=TREND_WINDOW)
    
    # Get feature matrix
    X = df_processed[feature_cols]
    
    return X, y_detrended, y_trend, y_original, feature_cols

def expanding_window_cv_splits(df, min_train_days, test_window):
    """Create expanding window splits"""
    n = len(df)
    splits = []
    train_end = min_train_days
    
    while train_end + test_window <= n:
        splits.append({
            'train_idx': list(range(0, train_end)),
            'test_idx': list(range(train_end, min(train_end + test_window, n)))
        })
        train_end += test_window
    
    return splits

def calculate_metrics(y_true, y_pred):
    """Calculate all metrics"""
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    # MAPE
    mask = y_true > 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan
    
    return {
        'r2': float(r2),
        'rmse': float(rmse),
        'mae': float(mae),
        'mape': float(mape)
    }

def train_and_evaluate(X_train, y_train, X_test, y_test, 
                       y_trend_train, y_trend_test, 
                       y_original_train, y_original_test, fold_num):
    """Train Ridge model and calculate train + test metrics"""
    
    # Standardize
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train model
    model = Ridge(alpha=ALPHA, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # TRAIN PREDICTIONS
    y_pred_train_detrended = model.predict(X_train_scaled)
    y_pred_train_absolute = y_trend_train.values + y_pred_train_detrended
    train_metrics = calculate_metrics(y_original_train.values, y_pred_train_absolute)
    
    # TEST PREDICTIONS
    y_pred_test_detrended = model.predict(X_test_scaled)
    y_pred_test_absolute = y_trend_test.values + y_pred_test_detrended
    test_metrics = calculate_metrics(y_original_test.values, y_pred_test_absolute)
    
    return {
        'fold': fold_num,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'train_r2': train_metrics['r2'],
        'train_rmse': train_metrics['rmse'],
        'train_mae': train_metrics['mae'],
        'train_mape': train_metrics['mape'],
        'test_r2': test_metrics['r2'],
        'test_rmse': test_metrics['rmse'],
        'test_mae': test_metrics['mae'],
        'test_mape': test_metrics['mape'],
        'overfitting_gap': train_metrics['r2'] - test_metrics['r2']
    }

def process_hotel(hotel_id):
    """Process single hotel with expanding window CV"""
    
    try:
        data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        
        if not data_file.exists():
            return {'hotel_id': hotel_id, 'status': 'missing_file'}
        
        df = pd.read_csv(data_file)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date').reset_index(drop=True)
        
        if len(df) < MIN_TRAIN_DAYS + TEST_WINDOW:
            return {'hotel_id': hotel_id, 'status': 'insufficient_data'}
        
        # Prepare features
        X_all, y_detrended, y_trend, y_original, feature_cols = prepare_features(df)
        
        if len(feature_cols) == 0:
            return {'hotel_id': hotel_id, 'status': 'no_features'}
        
        # Create CV splits
        cv_splits = expanding_window_cv_splits(df, MIN_TRAIN_DAYS, TEST_WINDOW)
        
        if len(cv_splits) == 0:
            return {'hotel_id': hotel_id, 'status': 'insufficient_cv_data'}
        
        # Run CV
        cv_results = []
        for fold_idx, split in enumerate(cv_splits, 1):
            X_train = X_all.iloc[split['train_idx']]
            y_train = y_detrended.iloc[split['train_idx']]
            X_test = X_all.iloc[split['test_idx']]
            y_test = y_detrended.iloc[split['test_idx']]
            y_trend_train = y_trend.iloc[split['train_idx']]
            y_trend_test = y_trend.iloc[split['test_idx']]
            y_original_train = y_original.iloc[split['train_idx']]
            y_original_test = y_original.iloc[split['test_idx']]
            
            fold_result = train_and_evaluate(
                X_train, y_train, X_test, y_test,
                y_trend_train, y_trend_test,
                y_original_train, y_original_test,
                fold_idx
            )
            cv_results.append(fold_result)
        
        # Calculate summary statistics
        train_r2_scores = [r['train_r2'] for r in cv_results]
        test_r2_scores = [r['test_r2'] for r in cv_results]
        
        cv_summary = {
            'mean_train_r2': float(np.mean(train_r2_scores)),
            'mean_test_r2': float(np.mean(test_r2_scores)),
            'median_test_r2': float(np.median(test_r2_scores)),
            'std_test_r2': float(np.std(test_r2_scores)),
            'min_test_r2': float(np.min(test_r2_scores)),
            'max_test_r2': float(np.max(test_r2_scores)),
            'mean_overfitting_gap': float(np.mean([r['overfitting_gap'] for r in cv_results])),
            'mean_test_rmse': float(np.mean([r['test_rmse'] for r in cv_results])),
            'mean_test_mape': float(np.mean([r['test_mape'] for r in cv_results]))
        }
        
        result = {
            'hotel_id': hotel_id,
            'status': 'success',
            'n_features': len(feature_cols),
            'n_observations': len(df),
            'cv_folds': len(cv_results),
            'fold_results': cv_results,
            'cv_summary': cv_summary
        }
        
        # Save individual hotel results
        with open(output_path / f'{hotel_id}_cv_results.json', 'w') as f:
            json.dump(result, f, indent=2)
        
        return result
        
    except Exception as e:
        return {
            'hotel_id': hotel_id,
            'status': 'error',
            'error': str(e)
        }

# Process all hotels
print("\nProcessing hotels with expanding window CV...")
print("-"*140)

all_results = {}
for idx, hotel_id in enumerate(hotel_list, 1):
    print(f"[{idx}/{len(hotel_list)}] {hotel_id}...", end='\r')
    result = process_hotel(hotel_id)
    all_results[hotel_id] = result

successful = [h for h, r in all_results.items() if r.get('status') == 'success']
print(f"\n\nSuccessfully processed: {len(successful)}/{len(hotel_list)} hotels")

# Generate detailed report
print("\n" + "="*140)
print("DETAILED RESULTS - OUT-OF-SAMPLE CROSS-VALIDATION")
print("="*140)

if len(successful) > 0:
    # Sort by mean test R²
    successful_sorted = sorted(successful, 
                              key=lambda h: all_results[h]['cv_summary']['mean_test_r2'], 
                              reverse=True)
    
    print("\n1. TOP 10 HOTELS - FOLD-BY-FOLD METRICS")
    print("="*140)
    
    for hotel_id in successful_sorted[:10]:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        print(f"\n{hotel_id.upper()}")
        print("-"*140)
        print(f"Observations: {result['n_observations']}, Features: {result['n_features']}, CV Folds: {result['cv_folds']}")
        print()
        print(f"{'Fold':<6} {'Train Size':<12} {'Test Size':<11} "
              f"{'Train R²':<11} {'Test R²':<11} {'Train RMSE':<12} {'Test RMSE':<12} "
              f"{'Train MAE':<11} {'Test MAE':<11} {'Train MAPE%':<12} {'Test MAPE%':<12} {'Gap':<10}")
        print("-"*140)
        
        for fold in result['fold_results']:
            print(f"{fold['fold']:<6} {fold['train_size']:<12} {fold['test_size']:<11} "
                  f"{fold['train_r2']:>10.4f} {fold['test_r2']:>10.4f} "
                  f"{fold['train_rmse']:>11.2f} {fold['test_rmse']:>11.2f} "
                  f"{fold['train_mae']:>10.2f} {fold['test_mae']:>10.2f} "
                  f"{fold['train_mape']:>11.2f} {fold['test_mape']:>11.2f} "
                  f"{fold['overfitting_gap']:>9.4f}")
        
        print()
        print(f"CV Summary: Mean Train R²={cv_sum['mean_train_r2']:.4f}, "
              f"Mean Test R²={cv_sum['mean_test_r2']:.4f}, "
              f"Std Test R²={cv_sum['std_test_r2']:.4f}, "
              f"Mean Gap={cv_sum['mean_overfitting_gap']:.4f}")
    
    # Summary table
    print("\n" + "="*140)
    print("2. SUMMARY TABLE - ALL HOTELS")
    print("="*140)
    print()
    print(f"{'Hotel':<12} {'Obs':<8} {'Features':<10} {'Folds':<7} "
          f"{'Mean Train R²':<15} {'Mean Test R²':<15} {'Std Test R²':<13} {'Mean Gap':<12}")
    print("-"*140)
    
    for hotel_id in successful_sorted:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        print(f"{hotel_id:<12} {result['n_observations']:<8} {result['n_features']:<10} {result['cv_folds']:<7} "
              f"{cv_sum['mean_train_r2']:>14.4f} {cv_sum['mean_test_r2']:>14.4f} "
              f"{cv_sum['std_test_r2']:>12.4f} {cv_sum['mean_overfitting_gap']:>11.4f}")
    
    # Performance distribution
    print("\n" + "="*140)
    print("3. PERFORMANCE DISTRIBUTION")
    print("="*140)
    
    test_r2_values = [all_results[h]['cv_summary']['mean_test_r2'] for h in successful]
    
    excellent = sum(1 for r2 in test_r2_values if r2 > 0.40)
    good = sum(1 for r2 in test_r2_values if 0.25 <= r2 <= 0.40)
    acceptable = sum(1 for r2 in test_r2_values if 0.15 <= r2 < 0.25)
    poor = sum(1 for r2 in test_r2_values if r2 < 0.15)
    total = len(test_r2_values)
    
    print()
    print(f"{'Category':<20} {'Count':<10} {'Percentage':<12}")
    print("-"*140)
    print(f"{'Excellent (R²>0.40)':<20} {excellent:<10} {100*excellent/total:>10.1f}%")
    print(f"{'Good (0.25-0.40)':<20} {good:<10} {100*good/total:>10.1f}%")
    print(f"{'Acceptable (0.15-0.25)':<20} {acceptable:<10} {100*acceptable/total:>10.1f}%")
    print(f"{'Poor (R²<0.15)':<20} {poor:<10} {100*poor/total:>10.1f}%")
    print(f"{'TOTAL':<20} {total:<10} {100.0:>10.1f}%")
    
    # Aggregate statistics
    print("\n" + "="*140)
    print("4. AGGREGATE STATISTICS")
    print("="*140)
    
    print(f"\nMean Test R²:     {np.mean(test_r2_values):.4f}")
    print(f"Median Test R²:   {np.median(test_r2_values):.4f}")
    print(f"Std Test R²:      {np.std(test_r2_values):.4f}")
    
    train_r2_values = [all_results[h]['cv_summary']['mean_train_r2'] for h in successful]
    print(f"\nMean Train R²:    {np.mean(train_r2_values):.4f}")
    print(f"Median Train R²:  {np.median(train_r2_values):.4f}")
    
    gap_values = [all_results[h]['cv_summary']['mean_overfitting_gap'] for h in successful]
    print(f"\nMean Overfitting Gap: {np.mean(gap_values):.4f}")
    print(f"Median Overfitting Gap: {np.median(gap_values):.4f}")
    
    # Export to CSV
    print("\n" + "="*140)
    print("5. EXPORTING RESULTS")
    print("="*140)
    
    # Hotel summary
    summary_records = []
    for hotel_id in successful:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        summary_records.append({
            'hotel_id': hotel_id,
            'n_observations': result['n_observations'],
            'n_features': result['n_features'],
            'cv_folds': result['cv_folds'],
            'mean_train_r2': cv_sum['mean_train_r2'],
            'mean_test_r2': cv_sum['mean_test_r2'],
            'median_test_r2': cv_sum['median_test_r2'],
            'std_test_r2': cv_sum['std_test_r2'],
            'min_test_r2': cv_sum['min_test_r2'],
            'max_test_r2': cv_sum['max_test_r2'],
            'mean_overfitting_gap': cv_sum['mean_overfitting_gap'],
            'mean_test_rmse': cv_sum['mean_test_rmse'],
            'mean_test_mape': cv_sum['mean_test_mape']
        })
    
    summary_df = pd.DataFrame(summary_records)
    summary_df = summary_df.sort_values('mean_test_r2', ascending=False)
    summary_df.to_csv(output_path / 'hotel_summary.csv', index=False)
    
    # Fold details
    fold_records = []
    for hotel_id in successful:
        result = all_results[hotel_id]
        for fold in result['fold_results']:
            fold_records.append({
                'hotel_id': hotel_id,
                'fold': fold['fold'],
                'train_size': fold['train_size'],
                'test_size': fold['test_size'],
                'train_r2': fold['train_r2'],
                'train_rmse': fold['train_rmse'],
                'train_mae': fold['train_mae'],
                'train_mape': fold['train_mape'],
                'test_r2': fold['test_r2'],
                'test_rmse': fold['test_rmse'],
                'test_mae': fold['test_mae'],
                'test_mape': fold['test_mape'],
                'overfitting_gap': fold['overfitting_gap']
            })
    
    fold_df = pd.DataFrame(fold_records)
    fold_df.to_csv(output_path / 'fold_details.csv', index=False)
    
    print(f"\n✓ Hotel summary: {output_path / 'hotel_summary.csv'}")
    print(f"✓ Fold details: {output_path / 'fold_details.csv'}")
    print(f"✓ Individual results: {output_path / '[hotel_id]_cv_results.json'}")

print("\n" + "="*140)
print("PROCESSING COMPLETE")
print("="*140)
print(f"Successful: {len(successful)}/{len(hotel_list)} hotels")
print(f"Results saved to: {output_path}/")

LINEAR REGRESSION + DETRENDING - OUT-OF-SAMPLE CROSS-VALIDATION
Model: Ridge Regression (α=100.0)
Preprocessing: 30-day detrending + StandardScaler
Features: Competitor prices + temporal features ONLY (NO own price lags)
CV Strategy: Expanding window (min 300 days, test window 50 days)
Testing 44 hotels
--------------------------------------------------------------------------------------------------------------------------------------------

Processing hotels with expanding window CV...
--------------------------------------------------------------------------------------------------------------------------------------------
[44/44] Hotel_44...

Successfully processed: 34/44 hotels

DETAILED RESULTS - OUT-OF-SAMPLE CROSS-VALIDATION

1. TOP 10 HOTELS - FOLD-BY-FOLD METRICS

HOTEL_35
--------------------------------------------------------------------------------------------------------------------------------------------
Observations: 393, Features: 50, CV Folds: 1

Fold   Train Size  

## LR with past data - Iteration2 - Log transformation 

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import warnings
warnings.filterwarnings('ignore')

# Configuration
processed_data_path = Path('../data/full-data/processed')
output_path = Path('../data/full-data/linear_log_detrending_oos_cv_clean')
output_path.mkdir(parents=True, exist_ok=True)

mapping_df = pd.read_csv('../data/full-data/hotel_mapping.csv')
hotel_list = mapping_df['masked_id'].tolist()

MIN_TRAIN_DAYS = 300
TEST_WINDOW = 50
TREND_WINDOW = 30
ALPHA = 100.0

print("="*140)
print("LINEAR + LOG + DETRENDING - NO OWN PRICE LAGS")
print("="*140)

def detrend_series_log(series, window=30):
    log_series = np.log1p(series.clip(lower=0))
    trend = log_series.rolling(window=window, min_periods=1, center=False).mean()
    detrended = log_series - trend
    return detrended, trend

def prepare_features(df):
    df_processed = df.copy()
    
    # Competitor columns only
    all_comp_cols = [col for col in df.columns 
                     if any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
    comp_base_cols = [col for col in all_comp_cols if '_lag_' not in col]
    comp_lag_cols = [col for col in all_comp_cols if '_lag_' in col]
    
    # Log + Detrend competitors
    for col in comp_base_cols:
        if col in df_processed.columns:
            detrended, _ = detrend_series_log(df_processed[col], window=TREND_WINDOW)
            df_processed[f'{col}_log_detrended'] = detrended
    
    for col in comp_lag_cols:
        if col in df_processed.columns:
            detrended, _ = detrend_series_log(df_processed[col], window=TREND_WINDOW)
            df_processed[f'{col}_log_detrended'] = detrended
    
    # Features
    feature_cols = []
    
    log_detrended_comp_lags = [f'{col}_log_detrended' for col in comp_lag_cols 
                               if f'{col}_log_detrended' in df_processed.columns]
    feature_cols.extend(log_detrended_comp_lags)
    
    temporal_cols = ['day_of_week', 'month', 'is_weekend', 'is_peak_season', 
                    'week_of_year', 'quarter', 'is_holiday_period', 'day_of_year',
                    'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
                    'sin_day_of_year', 'cos_day_of_year']
    temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
    feature_cols.extend(temporal_cols)
    
    # Target - log + detrend
    y_original = df_processed['base_rate']
    y_log_detrended, y_log_trend = detrend_series_log(y_original, window=TREND_WINDOW)
    
    X = df_processed[feature_cols]
    
    return X, y_log_detrended, y_log_trend, y_original, feature_cols

def expanding_window_cv_splits(df, min_train_days, test_window):
    n = len(df)
    splits = []
    train_end = min_train_days
    
    while train_end + test_window <= n:
        splits.append({
            'train_idx': list(range(0, train_end)),
            'test_idx': list(range(train_end, min(train_end + test_window, n)))
        })
        train_end += test_window
    
    return splits

def calculate_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    mask = y_true > 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan
    
    return {'r2': float(r2), 'rmse': float(rmse), 'mae': float(mae), 'mape': float(mape)}

def train_and_evaluate(X_train, y_train_log_detrended, X_test, y_test_log_detrended,
                       y_log_trend_train, y_log_trend_test,
                       y_original_train, y_original_test, fold_num):
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = Ridge(alpha=ALPHA, random_state=42)
    model.fit(X_train_scaled, y_train_log_detrended)
    
    # Train predictions
    y_pred_train_log_detrended = model.predict(X_train_scaled)
    y_pred_train_log = y_log_trend_train.values + y_pred_train_log_detrended
    y_pred_train_absolute = np.expm1(y_pred_train_log)
    train_metrics = calculate_metrics(y_original_train.values, y_pred_train_absolute)
    
    # Test predictions
    y_pred_test_log_detrended = model.predict(X_test_scaled)
    y_pred_test_log = y_log_trend_test.values + y_pred_test_log_detrended
    y_pred_test_absolute = np.expm1(y_pred_test_log)
    test_metrics = calculate_metrics(y_original_test.values, y_pred_test_absolute)
    
    return {
        'fold': fold_num,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'train_r2': train_metrics['r2'],
        'train_rmse': train_metrics['rmse'],
        'train_mae': train_metrics['mae'],
        'train_mape': train_metrics['mape'],
        'test_r2': test_metrics['r2'],
        'test_rmse': test_metrics['rmse'],
        'test_mae': test_metrics['mae'],
        'test_mape': test_metrics['mape'],
        'overfitting_gap': train_metrics['r2'] - test_metrics['r2']
    }

def process_hotel(hotel_id):
    try:
        data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        
        if not data_file.exists():
            return {'hotel_id': hotel_id, 'status': 'missing_file'}
        
        df = pd.read_csv(data_file)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date').reset_index(drop=True)
        
        if len(df) < MIN_TRAIN_DAYS + TEST_WINDOW:
            return {'hotel_id': hotel_id, 'status': 'insufficient_data'}
        
        X_all, y_log_detrended, y_log_trend, y_original, feature_cols = prepare_features(df)
        
        if len(feature_cols) == 0:
            return {'hotel_id': hotel_id, 'status': 'no_features'}
        
        cv_splits = expanding_window_cv_splits(df, MIN_TRAIN_DAYS, TEST_WINDOW)
        
        if len(cv_splits) == 0:
            return {'hotel_id': hotel_id, 'status': 'insufficient_cv_data'}
        
        cv_results = []
        for fold_idx, split in enumerate(cv_splits, 1):
            X_train = X_all.iloc[split['train_idx']]
            y_train = y_log_detrended.iloc[split['train_idx']]
            X_test = X_all.iloc[split['test_idx']]
            y_test = y_log_detrended.iloc[split['test_idx']]
            y_log_trend_train = y_log_trend.iloc[split['train_idx']]
            y_log_trend_test = y_log_trend.iloc[split['test_idx']]
            y_original_train = y_original.iloc[split['train_idx']]
            y_original_test = y_original.iloc[split['test_idx']]
            
            fold_result = train_and_evaluate(
                X_train, y_train, X_test, y_test,
                y_log_trend_train, y_log_trend_test,
                y_original_train, y_original_test,
                fold_idx
            )
            cv_results.append(fold_result)
        
        train_r2_scores = [r['train_r2'] for r in cv_results]
        test_r2_scores = [r['test_r2'] for r in cv_results]
        
        cv_summary = {
            'mean_train_r2': float(np.mean(train_r2_scores)),
            'mean_test_r2': float(np.mean(test_r2_scores)),
            'median_test_r2': float(np.median(test_r2_scores)),
            'std_test_r2': float(np.std(test_r2_scores)),
            'min_test_r2': float(np.min(test_r2_scores)),
            'max_test_r2': float(np.max(test_r2_scores)),
            'mean_overfitting_gap': float(np.mean([r['overfitting_gap'] for r in cv_results])),
            'mean_test_rmse': float(np.mean([r['test_rmse'] for r in cv_results])),
            'mean_test_mape': float(np.mean([r['test_mape'] for r in cv_results]))
        }
        
        result = {
            'hotel_id': hotel_id,
            'status': 'success',
            'n_features': len(feature_cols),
            'n_observations': len(df),
            'cv_folds': len(cv_results),
            'fold_results': cv_results,
            'cv_summary': cv_summary
        }
        
        with open(output_path / f'{hotel_id}_cv_results.json', 'w') as f:
            json.dump(result, f, indent=2)
        
        return result
        
    except Exception as e:
        return {'hotel_id': hotel_id, 'status': 'error', 'error': str(e)}

# Process all hotels
print("\nProcessing hotels...")
print("-"*140)

all_results = {}
for idx, hotel_id in enumerate(hotel_list, 1):
    print(f"[{idx}/{len(hotel_list)}] {hotel_id}...", end='\r')
    result = process_hotel(hotel_id)
    all_results[hotel_id] = result

successful = [h for h, r in all_results.items() if r.get('status') == 'success']
print(f"\n\nSuccessfully processed: {len(successful)}/{len(hotel_list)} hotels")

# Report
print("\n" + "="*140)
print("RESULTS")
print("="*140)

if len(successful) > 0:
    successful_sorted = sorted(successful, 
                              key=lambda h: all_results[h]['cv_summary']['mean_test_r2'], 
                              reverse=True)
    
    print("\nTOP 10 HOTELS")
    print("="*140)
    
    for hotel_id in successful_sorted[:10]:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        print(f"\n{hotel_id}: Obs={result['n_observations']}, Features={result['n_features']}, Folds={result['cv_folds']}")
        print(f"{'Fold':<6} {'Train Size':<12} {'Test Size':<11} {'Train R²':<11} {'Test R²':<11} "
              f"{'Train RMSE':<12} {'Test RMSE':<12} {'Gap':<10}")
        print("-"*140)
        
        for fold in result['fold_results']:
            print(f"{fold['fold']:<6} {fold['train_size']:<12} {fold['test_size']:<11} "
                  f"{fold['train_r2']:>10.4f} {fold['test_r2']:>10.4f} "
                  f"{fold['train_rmse']:>11.2f} {fold['test_rmse']:>11.2f} "
                  f"{fold['overfitting_gap']:>9.4f}")
        
        print(f"\nSummary: Mean Train R²={cv_sum['mean_train_r2']:.4f}, "
              f"Mean Test R²={cv_sum['mean_test_r2']:.4f}, Gap={cv_sum['mean_overfitting_gap']:.4f}")
    
    # Summary table
    print("\n" + "="*140)
    print("ALL HOTELS SUMMARY")
    print("="*140)
    print(f"{'Hotel':<12} {'Features':<10} {'Folds':<7} {'Mean Train R²':<15} "
          f"{'Mean Test R²':<15} {'Mean Gap':<12}")
    print("-"*140)
    
    for hotel_id in successful_sorted:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        print(f"{hotel_id:<12} {result['n_features']:<10} {result['cv_folds']:<7} "
              f"{cv_sum['mean_train_r2']:>14.4f} {cv_sum['mean_test_r2']:>14.4f} "
              f"{cv_sum['mean_overfitting_gap']:>11.4f}")
    
    # Stats
    test_r2_values = [all_results[h]['cv_summary']['mean_test_r2'] for h in successful]
    
    print("\nSTATISTICS")
    print("-"*140)
    print(f"Median Test R²: {np.median(test_r2_values):.4f}")
    print(f"Mean Test R²: {np.mean(test_r2_values):.4f}")
    
    excellent = sum(1 for r2 in test_r2_values if r2 > 0.40)
    poor = sum(1 for r2 in test_r2_values if r2 < 0.15)
    print(f"Excellent (R²>0.40): {excellent}/{len(successful)}")
    print(f"Poor (R²<0.15): {poor}/{len(successful)}")
    
    # Export
    summary_records = []
    fold_records = []
    
    for hotel_id in successful:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        summary_records.append({
            'hotel_id': hotel_id,
            'n_features': result['n_features'],
            'cv_folds': result['cv_folds'],
            'mean_train_r2': cv_sum['mean_train_r2'],
            'mean_test_r2': cv_sum['mean_test_r2'],
            'mean_overfitting_gap': cv_sum['mean_overfitting_gap']
        })
        
        for fold in result['fold_results']:
            fold_records.append({
                'hotel_id': hotel_id,
                'fold': fold['fold'],
                'train_r2': fold['train_r2'],
                'test_r2': fold['test_r2'],
                'overfitting_gap': fold['overfitting_gap']
            })
    
    pd.DataFrame(summary_records).to_csv(output_path / 'hotel_summary.csv', index=False)
    pd.DataFrame(fold_records).to_csv(output_path / 'fold_details.csv', index=False)
    
    print(f"\nFiles saved to: {output_path}/")

print("\nDONE")

LINEAR + LOG + DETRENDING - NO OWN PRICE LAGS

Processing hotels...
--------------------------------------------------------------------------------------------------------------------------------------------
[44/44] Hotel_44...

Successfully processed: 34/44 hotels

RESULTS

TOP 10 HOTELS

Hotel_35: Obs=393, Features=50, Folds=1
Fold   Train Size   Test Size   Train R²    Test R²     Train RMSE   Test RMSE    Gap       
--------------------------------------------------------------------------------------------------------------------------------------------
1      300          50              0.7960     0.7064       26.94       19.45    0.0895

Summary: Mean Train R²=0.7960, Mean Test R²=0.7064, Gap=0.0895

Hotel_27: Obs=425, Features=35, Folds=2
Fold   Train Size   Test Size   Train R²    Test R²     Train RMSE   Test RMSE    Gap       
--------------------------------------------------------------------------------------------------------------------------------------------
1      

## LR with past data - Iteration3 - Quantile Normalization

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import warnings
warnings.filterwarnings('ignore')

# Configuration
processed_data_path = Path('../data/full-data/processed')
output_path = Path('../data/full-data/linear_quantile_detrending_oos_cv_clean')
output_path.mkdir(parents=True, exist_ok=True)

mapping_df = pd.read_csv('../data/full-data/hotel_mapping.csv')
hotel_list = mapping_df['masked_id'].tolist()

MIN_TRAIN_DAYS = 300
TEST_WINDOW = 50
TREND_WINDOW = 30
ALPHA = 100.0

print("="*140)
print("LINEAR + QUANTILE + DETRENDING - NO OWN PRICE LAGS")
print("="*140)

def detrend_series(series, window=30):
    trend = series.rolling(window=window, min_periods=1, center=False).mean()
    detrended = series - trend
    return detrended, trend

def prepare_features(df):
    df_processed = df.copy()
    
    # Competitor columns only
    all_comp_cols = [col for col in df.columns 
                     if any(currency in col for currency in ['-USD', '-EUR', '-HKD', '-CNY'])]
    
    comp_base_cols = [col for col in all_comp_cols if '_lag_' not in col]
    comp_lag_cols = [col for col in all_comp_cols if '_lag_' in col]
    
    # Detrend competitors
    for col in comp_base_cols:
        if col in df_processed.columns:
            detrended, _ = detrend_series(df_processed[col], window=TREND_WINDOW)
            df_processed[f'{col}_detrended'] = detrended
    
    for col in comp_lag_cols:
        if col in df_processed.columns:
            detrended, _ = detrend_series(df_processed[col], window=TREND_WINDOW)
            df_processed[f'{col}_detrended'] = detrended
    
    # Features
    feature_cols = []
    
    detrended_comp_lags = [f'{col}_detrended' for col in comp_lag_cols 
                          if f'{col}_detrended' in df_processed.columns]
    feature_cols.extend(detrended_comp_lags)
    
    temporal_cols = ['day_of_week', 'month', 'is_weekend', 'is_peak_season', 
                    'week_of_year', 'quarter', 'is_holiday_period', 'day_of_year',
                    'sin_day_of_week', 'cos_day_of_week', 'sin_month', 'cos_month',
                    'sin_day_of_year', 'cos_day_of_year']
    temporal_cols = [col for col in temporal_cols if col in df_processed.columns]
    feature_cols.extend(temporal_cols)
    
    # Target
    y_original = df_processed['base_rate']
    y_detrended, y_trend = detrend_series(y_original, window=TREND_WINDOW)
    
    X = df_processed[feature_cols]
    
    return X, y_detrended, y_trend, y_original, feature_cols

def expanding_window_cv_splits(df, min_train_days, test_window):
    n = len(df)
    splits = []
    train_end = min_train_days
    
    while train_end + test_window <= n:
        splits.append({
            'train_idx': list(range(0, train_end)),
            'test_idx': list(range(train_end, min(train_end + test_window, n)))
        })
        train_end += test_window
    
    return splits

def calculate_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    mask = y_true > 0
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan
    
    return {'r2': float(r2), 'rmse': float(rmse), 'mae': float(mae), 'mape': float(mape)}

def train_and_evaluate(X_train, y_train, X_test, y_test, 
                       y_trend_train, y_trend_test, 
                       y_original_train, y_original_test, fold_num):
    
    # Quantile transform
    quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    X_train_quantile = quantile_transformer.fit_transform(X_train)
    X_test_quantile = quantile_transformer.transform(X_test)
    
    # Standardize after quantile
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_quantile)
    X_test_scaled = scaler.transform(X_test_quantile)
    
    model = Ridge(alpha=ALPHA, random_state=42)
    model.fit(X_train_scaled, y_train)
    
    # Train predictions
    y_pred_train_detrended = model.predict(X_train_scaled)
    y_pred_train_absolute = y_trend_train.values + y_pred_train_detrended
    train_metrics = calculate_metrics(y_original_train.values, y_pred_train_absolute)
    
    # Test predictions
    y_pred_test_detrended = model.predict(X_test_scaled)
    y_pred_test_absolute = y_trend_test.values + y_pred_test_detrended
    test_metrics = calculate_metrics(y_original_test.values, y_pred_test_absolute)
    
    return {
        'fold': fold_num,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'train_r2': train_metrics['r2'],
        'train_rmse': train_metrics['rmse'],
        'train_mae': train_metrics['mae'],
        'train_mape': train_metrics['mape'],
        'test_r2': test_metrics['r2'],
        'test_rmse': test_metrics['rmse'],
        'test_mae': test_metrics['mae'],
        'test_mape': test_metrics['mape'],
        'overfitting_gap': train_metrics['r2'] - test_metrics['r2']
    }

def process_hotel(hotel_id):
    try:
        data_file = processed_data_path / f'{hotel_id}_lagged_dataset.csv'
        
        if not data_file.exists():
            return {'hotel_id': hotel_id, 'status': 'missing_file'}
        
        df = pd.read_csv(data_file)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date').reset_index(drop=True)
        
        if len(df) < MIN_TRAIN_DAYS + TEST_WINDOW:
            return {'hotel_id': hotel_id, 'status': 'insufficient_data'}
        
        X_all, y_detrended, y_trend, y_original, feature_cols = prepare_features(df)
        
        if len(feature_cols) == 0:
            return {'hotel_id': hotel_id, 'status': 'no_features'}
        
        cv_splits = expanding_window_cv_splits(df, MIN_TRAIN_DAYS, TEST_WINDOW)
        
        if len(cv_splits) == 0:
            return {'hotel_id': hotel_id, 'status': 'insufficient_cv_data'}
        
        cv_results = []
        for fold_idx, split in enumerate(cv_splits, 1):
            X_train = X_all.iloc[split['train_idx']]
            y_train = y_detrended.iloc[split['train_idx']]
            X_test = X_all.iloc[split['test_idx']]
            y_test = y_detrended.iloc[split['test_idx']]
            y_trend_train = y_trend.iloc[split['train_idx']]
            y_trend_test = y_trend.iloc[split['test_idx']]
            y_original_train = y_original.iloc[split['train_idx']]
            y_original_test = y_original.iloc[split['test_idx']]
            
            fold_result = train_and_evaluate(
                X_train, y_train, X_test, y_test,
                y_trend_train, y_trend_test,
                y_original_train, y_original_test,
                fold_idx
            )
            cv_results.append(fold_result)
        
        train_r2_scores = [r['train_r2'] for r in cv_results]
        test_r2_scores = [r['test_r2'] for r in cv_results]
        
        cv_summary = {
            'mean_train_r2': float(np.mean(train_r2_scores)),
            'mean_test_r2': float(np.mean(test_r2_scores)),
            'median_test_r2': float(np.median(test_r2_scores)),
            'std_test_r2': float(np.std(test_r2_scores)),
            'min_test_r2': float(np.min(test_r2_scores)),
            'max_test_r2': float(np.max(test_r2_scores)),
            'mean_overfitting_gap': float(np.mean([r['overfitting_gap'] for r in cv_results])),
            'mean_test_rmse': float(np.mean([r['test_rmse'] for r in cv_results])),
            'mean_test_mape': float(np.mean([r['test_mape'] for r in cv_results]))
        }
        
        result = {
            'hotel_id': hotel_id,
            'status': 'success',
            'n_features': len(feature_cols),
            'n_observations': len(df),
            'cv_folds': len(cv_results),
            'fold_results': cv_results,
            'cv_summary': cv_summary
        }
        
        with open(output_path / f'{hotel_id}_cv_results.json', 'w') as f:
            json.dump(result, f, indent=2)
        
        return result
        
    except Exception as e:
        return {'hotel_id': hotel_id, 'status': 'error', 'error': str(e)}

# Process all hotels
print("\nProcessing hotels...")
print("-"*140)

all_results = {}
for idx, hotel_id in enumerate(hotel_list, 1):
    print(f"[{idx}/{len(hotel_list)}] {hotel_id}...", end='\r')
    result = process_hotel(hotel_id)
    all_results[hotel_id] = result

successful = [h for h, r in all_results.items() if r.get('status') == 'success']
print(f"\n\nSuccessfully processed: {len(successful)}/{len(hotel_list)} hotels")

# Report
print("\n" + "="*140)
print("RESULTS")
print("="*140)

if len(successful) > 0:
    successful_sorted = sorted(successful, 
                              key=lambda h: all_results[h]['cv_summary']['mean_test_r2'], 
                              reverse=True)
    
    print("\nTOP 10 HOTELS")
    print("="*140)
    
    for hotel_id in successful_sorted[:10]:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        print(f"\n{hotel_id}: Obs={result['n_observations']}, Features={result['n_features']}, Folds={result['cv_folds']}")
        print(f"{'Fold':<6} {'Train Size':<12} {'Test Size':<11} {'Train R²':<11} {'Test R²':<11} "
              f"{'Train RMSE':<12} {'Test RMSE':<12} {'Gap':<10}")
        print("-"*140)
        
        for fold in result['fold_results']:
            print(f"{fold['fold']:<6} {fold['train_size']:<12} {fold['test_size']:<11} "
                  f"{fold['train_r2']:>10.4f} {fold['test_r2']:>10.4f} "
                  f"{fold['train_rmse']:>11.2f} {fold['test_rmse']:>11.2f} "
                  f"{fold['overfitting_gap']:>9.4f}")
        
        print(f"\nSummary: Mean Train R²={cv_sum['mean_train_r2']:.4f}, "
              f"Mean Test R²={cv_sum['mean_test_r2']:.4f}, Gap={cv_sum['mean_overfitting_gap']:.4f}")
    
    # Summary table
    print("\n" + "="*140)
    print("ALL HOTELS SUMMARY")
    print("="*140)
    print(f"{'Hotel':<12} {'Features':<10} {'Folds':<7} {'Mean Train R²':<15} "
          f"{'Mean Test R²':<15} {'Mean Gap':<12}")
    print("-"*140)
    
    for hotel_id in successful_sorted:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        print(f"{hotel_id:<12} {result['n_features']:<10} {result['cv_folds']:<7} "
              f"{cv_sum['mean_train_r2']:>14.4f} {cv_sum['mean_test_r2']:>14.4f} "
              f"{cv_sum['mean_overfitting_gap']:>11.4f}")
    
    # Stats
    test_r2_values = [all_results[h]['cv_summary']['mean_test_r2'] for h in successful]
    
    print("\nSTATISTICS")
    print("-"*140)
    print(f"Median Test R²: {np.median(test_r2_values):.4f}")
    print(f"Mean Test R²: {np.mean(test_r2_values):.4f}")
    
    excellent = sum(1 for r2 in test_r2_values if r2 > 0.40)
    poor = sum(1 for r2 in test_r2_values if r2 < 0.15)
    print(f"Excellent (R²>0.40): {excellent}/{len(successful)}")
    print(f"Poor (R²<0.15): {poor}/{len(successful)}")
    
    # Export
    summary_records = []
    fold_records = []
    
    for hotel_id in successful:
        result = all_results[hotel_id]
        cv_sum = result['cv_summary']
        
        summary_records.append({
            'hotel_id': hotel_id,
            'n_features': result['n_features'],
            'cv_folds': result['cv_folds'],
            'mean_train_r2': cv_sum['mean_train_r2'],
            'mean_test_r2': cv_sum['mean_test_r2'],
            'mean_overfitting_gap': cv_sum['mean_overfitting_gap']
        })
        
        for fold in result['fold_results']:
            fold_records.append({
                'hotel_id': hotel_id,
                'fold': fold['fold'],
                'train_r2': fold['train_r2'],
                'test_r2': fold['test_r2'],
                'overfitting_gap': fold['overfitting_gap']
            })
    
    pd.DataFrame(summary_records).to_csv(output_path / 'hotel_summary.csv', index=False)
    pd.DataFrame(fold_records).to_csv(output_path / 'fold_details.csv', index=False)
    
    print(f"\nFiles saved to: {output_path}/")

print("\nDONE")

LINEAR + QUANTILE + DETRENDING - NO OWN PRICE LAGS

Processing hotels...
--------------------------------------------------------------------------------------------------------------------------------------------
[44/44] Hotel_44...

Successfully processed: 34/44 hotels

RESULTS

TOP 10 HOTELS

Hotel_35: Obs=393, Features=50, Folds=1
Fold   Train Size   Test Size   Train R²    Test R²     Train RMSE   Test RMSE    Gap       
--------------------------------------------------------------------------------------------------------------------------------------------
1      300          50              0.7796     0.6673       28.00       20.71    0.1123

Summary: Mean Train R²=0.7796, Mean Test R²=0.6673, Gap=0.1123

Hotel_40: Obs=441, Features=45, Folds=2
Fold   Train Size   Test Size   Train R²    Test R²     Train RMSE   Test RMSE    Gap       
--------------------------------------------------------------------------------------------------------------------------------------------
1 